In [67]:
# The aim of this script is to calculate the rental yield by suburb
# then to identify arbtriage opportunities to rent, live in or invest 
# the dataset produced on the back of this process can be used to analyse market gaps and biasa

In [68]:
import pandas as pd

# R_INFO_ACT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_ACT.pkl') 
# A_INFO_ACT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_ACT.pkl') 

# R_INFO_NSW = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_NSW.pkl') 
# A_INFO_NSW = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_NSW.pkl') 

# R_INFO_NT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_NT.pkl') 
# A_INFO_NT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_NT.pkl') 

# R_INFO_QLD = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_QLD.pkl') 
# A_INFO_QLD = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_QLD.pkl') 

# R_INFO_SA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_SA.pkl') 
# A_INFO_SA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_SA.pkl') 

# R_INFO_TAS = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_TAS.pkl') 
# A_INFO_TAS = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_TAS.pkl') 

# R_INFO_VIC = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_VIC.pkl') 
# A_INFO_VIC = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_VIC.pkl') 

R_INFO_WA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_RENT_WA.pkl') 
A_INFO_WA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_WA.pkl')


# Append state based files
R_INFO_ALL = pd.DataFrame()
A_INFO_ALL = pd.DataFrame()

INFO_RENT = R_INFO_ALL.append([R_INFO_WA],ignore_index=True).reset_index()
INFO_ALL = A_INFO_ALL.append([A_INFO_WA],ignore_index=True).reset_index() 


# INFO_RENT = R_INFO_ALL.append([R_INFO_ACT, R_INFO_NSW, R_INFO_NT, R_INFO_QLD, R_INFO_SA, R_INFO_TAS, R_INFO_VI, R_INFO_WA],ignore_index=True).reset_index() 
# INFO_ALL = A_INFO_ALL.append([A_INFO_ACT, A_INFO_NSW, A_INFO_NT, A_INFO_QLD, A_INFO_SA, A_INFO_TAS, A_INFO_VIC, A_INFO_WA],ignore_index=True).reset_index() 

del R_INFO_WA, R_INFO_ALL
del A_INFO_WA, A_INFO_ALL

# del R_INFO_ALL, R_INFO_ACT, R_INFO_NSW, R_INFO_NT, R_INFO_QLD, R_INFO_SA, R_INFO_TAS, R_INFO_VIC, R_INFO_WA
# del A_INFO_ALL, A_INFO_ACT, A_INFO_NSW, A_INFO_NT, A_INFO_QLD, A_INFO_SA, A_INFO_TAS, A_INFO_VIC, A_INFO_WA

# temp download file for testing
# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/test_extract_rent_WA{}.xlsx".format(dte)
# I_INFO.to_excel(fname)


# create masterlist of postcode and suburbs

REF_PCODE_SBRB = INFO_ALL.append([INFO_ALL],ignore_index=True).reset_index() 
# REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb"]]
# full comparison version
REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb","property_type","bedrooms","bathrooms"]]
REF_PCODE_SBRB = REF_PCODE_SBRB.drop_duplicates()


# need to remove rentals from the ALL masterlist


In [69]:
# Remove rentals from the masterlist
A_FIND_BUY = INFO_ALL.merge(INFO_RENT,how='outer',on=["postcode","suburb","property_type","bedrooms","bathrooms"], indicator= True)
INFO_BUY = A_FIND_BUY[A_FIND_BUY['_merge']=="left_only"]
INFO_BUY = INFO_BUY.reset_index()
INFO_BUY.columns = INFO_BUY.columns.str.replace("_x","")
# del A_FIND_BUY

In [70]:
# find exact match suburbs
# from soupsieve import match


INFO_RENT_ = INFO_RENT[['full_address','price']]
INFO_BUY_ = INFO_BUY[['full_address','price']]

INFO_RENT_ADDR = INFO_RENT_.groupby(["full_address"])["price"].count().reset_index()
INFO_BUY_ADDR = INFO_BUY_.groupby(["full_address"])["price"].count().reset_index()

INFO_RENT_ADDR.rename(columns={'price':'num_rent'}, inplace = True)
INFO_BUY_ADDR.rename(columns={'price':'num_all'}, inplace = True)

dp_comp_exact = INFO_RENT_ADDR.merge(INFO_BUY_ADDR,how='inner',on=["full_address"])

# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/REALESTATE_BUY_AND_RENT{}.xlsx".format(dte)
# dp_comp_exact.to_excel(fname)

del INFO_RENT_, INFO_BUY_, INFO_RENT_ADDR, INFO_BUY_ADDR

In [71]:
# Rental data
INFO_RENT_1 = INFO_RENT[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)

# INFO_RENT_1 = R_INFO_ALL.drop(columns=['badge','url','listing_company_name','listing_company_phone','description']).fillna(0)
INFO_RENT_1 = INFO_RENT_1.drop_duplicates()
# summarise by postcode and suburb

# INFO_RENT_MED = INFO_RENT_1.groupby(["postcode","suburb"])["price"].median().reset_index()
INFO_RENT_MED = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms","bathrooms"])["price"].median().reset_index()
INFO_RENT_MED.rename(columns={'price':'price_RENT'}, inplace = True)

# INFO_RENT_FREQ = INFO_RENT_1.groupby(["postcode","suburb"])["full_address"].count().reset_index()
INFO_RENT_FREQ = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms","bathrooms"])["full_address"].count().reset_index()
INFO_RENT_FREQ.rename(columns={'full_address':'num_RENT'}, inplace = True)

# del INFO_RENT_1 

In [72]:
# Sales data
INFO_BUY_1 = INFO_BUY[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)
# A_INFO_SUBSET= INFO_BUY_.reset_index()
INFO_BUY_1.drop_duplicates()
# summarise by postcode and suburb

# INFO_BUY_MED = INFO_BUY_1.groupby(["postcode","suburb"])["price"].median().reset_index()
INFO_BUY_MED = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms","bathrooms"])["price"].median().reset_index()
INFO_BUY_MED.rename(columns={'price':'price_BUY'}, inplace = True)

# INFO_BUY_FREQ = INFO_BUY_1.groupby(["postcode","suburb"])["full_address"].count().reset_index()
INFO_BUY_FREQ = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms","bathrooms"])["full_address"].count().reset_index()
INFO_BUY_FREQ.rename(columns={'full_address':'num_BUY'}, inplace = True)

# del INFO_BUY_1

In [73]:
df_comp = REF_PCODE_SBRB.merge(INFO_BUY_MED,how='left',on=["postcode","suburb","property_type","bedrooms","bathrooms"]).merge(INFO_RENT_MED,how='left',on=["postcode","suburb","property_type","bedrooms","bathrooms"]).fillna(0)
df_comp = df_comp.merge(INFO_BUY_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms","bathrooms"]).merge(INFO_RENT_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms","bathrooms"]).fillna(0)

df_comp.dtypes
# df_comp = REF_PCODE_SBRB.merge(R_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(R_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)
# df_comp = df_comp.merge(A_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(A_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)

# if pd.to_numeric(df_comp['price_RENT']) == 0:
    # df_comp['YEARLY_RENT']  = 0
    # df_comp['RENT_YIELD']   = 0
    # df_comp['RENT_DIV_BUY'] = 0
# else:
df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']
df_comp['RENT_DIV_BUY'] = df_comp['num_RENT'] / df_comp['num_BUY']



        # if INF['dividendRate'] is not None:
        #     INF['dr']=INF['dividendRate']
        # else:
        #     INF['dr']=0


# create combo key
# df_comp['Property Desc'] = df_comp['postcode']+' ('+df_comp['suburb']+') :'+df_comp['property_type']+' '+df_comp['property_type']+str(df_comp['bedrooms'])+'bedroom '+str(df_comp['bathrooms'])+'baths'

# del R_INFO_SUBSET_MED, R_INFO_SUBSET_FREQ, A_INFO_SUBSET_MED, A_INFO_SUBSET_FREQ
# 7303

In [74]:
# temp download file for testing
from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/test_extract_rent_WA{}.xlsx".format(dte)
INFO_RENT_1.to_excel(fname)

fname = "/Users/joezhou/Downloads/test_extract_BUY_WA{}.xlsx".format(dte)
INFO_BUY_1.to_excel(fname)

fname = "/Users/joezhou/Downloads/test_extract_ALL_summary{}.xlsx".format(dte)
df_comp.to_excel(fname)
